In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from textblob import TextBlob
from googletrans import Translator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pyvi import ViTokenizer, ViUtils


In [8]:
# Đường dẫn tới file CSV
file_path = r".\sentiments_v2.txt"
data = pd.read_csv(file_path, on_bad_lines='skip')  # Sử dụng on_bad_lines='skip' thay cho error_bad_lines=False
print(data.head())  # Hiển thị 5 bản ghi đầu tiên

                                                                                                                                                                     userid\tmtime\trating_star\tcomment
69545822\t20-05-2021 11:29:58\t5\tĐồ giống hình     nhìn khá xinh                                      giá rẻ                                           ship siêu nhanh luôn. Chưa dùng thử nên chưa ...
49021104\t08-05-2021 22:10:31\t5\tGiao hàng nha...  shop bán hàng giá rẻ và rất oke luôn. Nhìn màu...  có cơ hội thì mình cũng sẽ ủng hộ                                                             NaN
275113953\t30-06-2021 21:42:59\t5\tSản phẩm giố... NaN                                                NaN                                                                                            NaN
470905623\t24-06-2021 11:41:08\t5\tSon xinh xĩu...  lên màu đẹp                                        với giá rẻ như vạy thì ko cần phải bàn nữa rồi                                               

In [18]:
# Đọc file CSV
file_path = "sentiments_v2.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print(data.columns)

Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')


In [21]:
# Đọc file CSV
file_path = "sentiments_v2.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Chỉ kiểm tra nếu cột 'comment' tồn tại
if 'comment' in data.columns:
    # Khởi tạo công cụ dịch
    translator = Translator()

    # Hàm dịch và phân tích cảm xúc
    def analyze_sentiment(text):
        # Dịch văn bản sang tiếng Anh
        translated_text = translator.translate(text, src='vi', dest='en').text
        # Phân tích cảm xúc bằng TextBlob
        blob = TextBlob(translated_text)
        return blob.sentiment.polarity  # Trả về điểm cảm xúc từ -1 (tiêu cực) đến 1 (tích cực)

    # Áp dụng hàm cho 5 bình luận đầu tiên
    data['sentiment'] = data['comment'].head(5).apply(analyze_sentiment)

    # Hiển thị 5 bình luận đầu tiên với điểm cảm xúc
    print(data[['comment', 'sentiment']].head(5))
else:
    print("Cột 'comment' không tồn tại trong DataFrame.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')
                                             comment  sentiment
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   0.295833
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   0.391667
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  -0.025000
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   0.850000
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  -0.125000
